In [1]:
import pandas as pd
import xgboost as xgb
import os
import logging
import time
import cPickle as pickle
import numpy as np
import sys

from IPython.utils.io import Tee
wtf = Tee('../logs/%s.log' % ('xgbenc' + str(int(time.time()))), 'w', channel='stderr')

In [ ]:
%%time
PATH_STORE = '../saved/storage.h5'
train_pd = pd.read_hdf(PATH_STORE, 'train')

In [ ]:
%%time
# Out-of-memory
PATH_LIBSVM = '/tmp/train.libsvm'
PATH_CACHE = '/tmp/dtrain.cache'
if os.path.exists(PATH_CACHE):
    print 'Remove old cache @ %s' % PATH_CACHE
    os.remove(PATH_CACHE)
    os.remove(PATH_CACHE + '.row.blob')

# dtrain = xgb.DMatrix('../saved/med.libsvm#%s' % PATH_CACHE)
dtrain = xgb.DMatrix('%s#%s' % (PATH_LIBSVM, PATH_CACHE))

In [2]:
%%time
# In-memory
# PATH_LIBSVM = '/tmp/train.libsvm'
# PATH_LIBSVM = '/tmp/train_flagged.libsvm'
PATH_LIBSVM = '/tmp/train_xgbenc.libsvm'
dtrain = xgb.DMatrix(PATH_LIBSVM)

CPU times: user 1min 27s, sys: 1.82 s, total: 1min 29s
Wall time: 29.9 s


In [3]:
params = {}
params['objective'] = "binary:logistic"
params['eta'] = 0.01
params['min_child_weight'] = 6
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['max_delta_step'] = 1
params['silent'] = 1
params['max_depth'] = 9 #7
params['eval_metric'] = 'auc'
# params["scale_pos_weight"] = 1.0
params['gamma'] = 5
params['alpha'] = 1
# params['nthread'] = 4

def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label==1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

plst = list(params.items())

max_rounds = 7000


normal feats: [4000] 0.790278

numflag feats: [4940]	cv-test-auc:0.790382+0.001484	cv-train-auc:0.999978+0.000003

In [9]:
%%time
# CV
cv_log = xgb.cv(params, dtrain, max_rounds, nfold=3,
                metrics={'auc'}, seed = 0,
                fpreproc=fpreproc)

pickle.dump((params, cv_log), open('../saved/cv_log_%d.p' % int(time.time()), 'wb'))

[0]	cv-test-auc:0.695887+0.003349	cv-train-auc:0.758380+0.000838
[1]	cv-test-auc:0.722475+0.004446	cv-train-auc:0.791200+0.004035
[2]	cv-test-auc:0.732576+0.003201	cv-train-auc:0.803913+0.004263
[3]	cv-test-auc:0.738319+0.003777	cv-train-auc:0.811902+0.004477
[4]	cv-test-auc:0.741789+0.003345	cv-train-auc:0.817480+0.003936
[5]	cv-test-auc:0.745013+0.003245	cv-train-auc:0.821624+0.003697
[6]	cv-test-auc:0.746905+0.003710	cv-train-auc:0.824767+0.003293
[7]	cv-test-auc:0.747766+0.003703	cv-train-auc:0.826350+0.003470
[8]	cv-test-auc:0.749421+0.003150	cv-train-auc:0.828579+0.003439
[9]	cv-test-auc:0.750226+0.003373	cv-train-auc:0.829913+0.003377
[10]	cv-test-auc:0.751103+0.002874	cv-train-auc:0.831403+0.003206
[11]	cv-test-auc:0.752016+0.002518	cv-train-auc:0.832407+0.003221
[12]	cv-test-auc:0.752734+0.002771	cv-train-auc:0.833777+0.003344
[13]	cv-test-auc:0.753213+0.002626	cv-train-auc:0.834555+0.003433
[14]	cv-test-auc:0.753778+0.002620	cv-train-auc:0.835281+0.003097
[15]	cv-test-auc:0.7

KeyboardInterrupt: 

In [4]:
enc_ind = np.load('../saved/stupid_encode/enc_ind.npy')
val_ind = np.load('../saved/stupid_encode/val_ind.npy')

In [5]:
dtrain.num_row()

145175L

In [ ]:
# Train with eval
watchlist = [
#     (dtrain.slice(enc_ind), 'train'),
    (dtrain.slice(val_ind), 'val'),
]
bst = xgb.train(params, dtrain.slice(enc_ind), max_rounds, 
                evals=watchlist, 
                early_stopping_rounds=200, 
                verbose_eval=True)

In [13]:
bst.save_model('xgbenc_%d.model' % int(time.time()))

In [5]:
y = dtrain.get_label()

In [6]:
from collections import Counter
Counter(y)

Counter({0.0: 111415, 1.0: 33760})

In [12]:
for l in cv_log[-2000::100]:
    print l

[4000]	cv-test-auc:0.790285+0.001357	cv-train-auc:0.999898+0.000003
[4100]	cv-test-auc:0.790348+0.001374	cv-train-auc:0.999915+0.000001
[4200]	cv-test-auc:0.790372+0.001395	cv-train-auc:0.999930+0.000001
[4300]	cv-test-auc:0.790378+0.001396	cv-train-auc:0.999941+0.000001
[4400]	cv-test-auc:0.790408+0.001415	cv-train-auc:0.999951+0.000001
[4500]	cv-test-auc:0.790441+0.001425	cv-train-auc:0.999958+0.000001
[4600]	cv-test-auc:0.790460+0.001394	cv-train-auc:0.999964+0.000000
[4700]	cv-test-auc:0.790479+0.001412	cv-train-auc:0.999969+0.000000
[4800]	cv-test-auc:0.790457+0.001434	cv-train-auc:0.999974+0.000001
[4900]	cv-test-auc:0.790447+0.001440	cv-train-auc:0.999977+0.000001
[5000]	cv-test-auc:0.790485+0.001433	cv-train-auc:0.999981+0.000001
[5100]	cv-test-auc:0.790490+0.001441	cv-train-auc:0.999983+0.000001
[5200]	cv-test-auc:0.790487+0.001445	cv-train-auc:0.999986+0.000001
[5300]	cv-test-auc:0.790517+0.001425	cv-train-auc:0.999987+0.000001
[5400]	cv-test-auc:0.790537+0.001444	cv-train-au

# Train on whole set

In [ ]:
bst = xgb.train(params, dtrain, max_rounds)
bst.save_model('%d.model' % int(time.time()))

# Predict

In [2]:
bst = xgb.Booster(model_file='../notebooks/xgbenc_1443146969.model')

In [3]:
# In-memory
# PATH_TEST_LIBSVM = '/tmp/test.libsvm'
# PATH_TEST_LIBSVM = '/tmp/test_flagged.libsvm'
PATH_TEST_LIBSVM = '/tmp/test_xgbenc.libsvm'
dtest = xgb.DMatrix(PATH_TEST_LIBSVM)

In [4]:
preds = bst.predict(dtest)

# Create submission

In [5]:
sub = pd.read_csv('../data/sample_submission.csv', index_col='ID')
sub['target'] = preds
sub.to_csv('../submissions/xgb_encoded.csv')

# Scratchspace

In [18]:
import gc
del dtrain
gc.collect()

16

In [9]:
sys

NameError: name 'sys' is not defined